In [2]:
import optuna
import pickle
import torch
import os
import sys
import ROOT
import numpy as np
import matplotlib.pyplot as plt
import yaml
import pandas as pd
from matplotlib.ticker import MultipleLocator
from scipy.optimize import linear_sum_assignment


# Add the root folder to Python path
root_folder = os.path.abspath(os.path.join(os.getcwd(), ".."))
if root_folder not in sys.path:
    sys.path.append(root_folder)

import lib.unet_nn as UNet
import lib.modified_aggregation as MA
from lib.modified_aggregation_clusterer import ModifiedAggregationClusterer
from lib.unet_clusterer import UNetClusterer
from lib.focal import FocalH
from lib.base_nn import Data
from lib import metrics# import count_clusters, count_labels, compute_score, total, separation_efficiency
from lib.misc_util import open_bundle

os.chdir('/home/bjartur/workspace/python_focalh_clustering/') # Laptop and Desktop

# Inspect files

In [10]:
file = open_bundle("evaluation/best/cnn/eval_cnn_train_tiny_eval_calib.pkl")
file["data"]

{'name': 'train_tiny',
 'files': [{'path': 'data/train/TRAIN_E350_P1_N10.root',
   'particles': 1,
   'saturation': 4095,
   'meta': {'source': 'mc', 'detector': 'prototype2'}},
  {'path': 'data/train/TRAIN_E350_P2_N10.root',
   'particles': 2,
   'saturation': 4095,
   'meta': {'source': 'mc', 'detector': 'prototype2'}},
  {'path': 'data/train/TRAIN_E350_P3_N10.root',
   'particles': 3,
   'saturation': 4095,
   'meta': {'source': 'mc', 'detector': 'prototype2'}},
  {'path': 'data/train/TRAIN_E350_P4_N10.root',
   'particles': 4,
   'saturation': 4095,
   'meta': {'source': 'mc', 'detector': 'prototype2'}},
  {'path': 'data/train/TRAIN_E350_P5_N10.root',
   'particles': 5,
   'saturation': 4095,
   'meta': {'source': 'mc', 'detector': 'prototype2'}},
  {'path': 'data/train/TRAIN_E350_P6_N10.root',
   'particles': 6,
   'saturation': 4095,
   'meta': {'source': 'mc', 'detector': 'prototype2'}},
  {'path': 'data/train/TRAIN_E350_P7_N10.root',
   'particles': 7,
   'saturation': 4095,
  

In [5]:
for k,v in file.items():
    print(k)

method
study
data
its
timestamps
load_path
eval


In [8]:
file["data"]

{'name': 'train_tiny',
 'files': [{'path': 'data/train/TRAIN_E350_P1_N10.root',
   'particles': 1,
   'saturation': 4095,
   'meta': {'source': 'mc', 'detector': 'prototype2'}},
  {'path': 'data/train/TRAIN_E350_P2_N10.root',
   'particles': 2,
   'saturation': 4095,
   'meta': {'source': 'mc', 'detector': 'prototype2'}},
  {'path': 'data/train/TRAIN_E350_P3_N10.root',
   'particles': 3,
   'saturation': 4095,
   'meta': {'source': 'mc', 'detector': 'prototype2'}},
  {'path': 'data/train/TRAIN_E350_P4_N10.root',
   'particles': 4,
   'saturation': 4095,
   'meta': {'source': 'mc', 'detector': 'prototype2'}},
  {'path': 'data/train/TRAIN_E350_P5_N10.root',
   'particles': 5,
   'saturation': 4095,
   'meta': {'source': 'mc', 'detector': 'prototype2'}},
  {'path': 'data/train/TRAIN_E350_P6_N10.root',
   'particles': 6,
   'saturation': 4095,
   'meta': {'source': 'mc', 'detector': 'prototype2'}},
  {'path': 'data/train/TRAIN_E350_P7_N10.root',
   'particles': 7,
   'saturation': 4095,
  